<a href="https://colab.research.google.com/github/shahhassansh/UCF_CAP5610_ML_Assignments/blob/master/Problem5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Problem 5**

###**Importing Libraries**

In [0]:
from keras.datasets import mnist
from keras.utils import to_categorical
from keras import models
from keras import layers
import numpy as np

Using TensorFlow backend.


###**Loading Data**

In [0]:
(train_images_raw, train_labels_raw), (test_images_raw, test_labels_raw) = mnist.load_data()

11493376/11490434 [==============================] - 7s 1us/step


###**Preparing Images for Number of Region Calculation** 

In [0]:
train_images = np.zeros((60000,28,28))
for counter, a in enumerate(train_images_raw):
  for j, b in enumerate(a):
    for k,c in enumerate(b):
      if (c==0):
        train_images[counter,j,k] = 1
      elif (c!=0):
        train_images[counter,j,k] = 0
      

###**Program for Number of Region Calculation**

In [0]:
class Graph: 
  def __init__(self, row, col, g): 
    self.ROW = row 
    self.COL = col 
    self.graph = g
  def isSafe(self, i, j, visited):
    return (i >= 0 and i < self.ROW and j >= 0 and j < self.COL and not visited[i][j] and self.graph[i][j])            
  def DFS(self, i, j, visited):
    rowNbr = [-1, -1, -1,  0, 0,  1, 1, 1]; 
    colNbr = [-1,  0,  1, -1, 1, -1, 0, 1]; 
    visited[i][j] = True
    for k in range(8):
      if self.isSafe(i + rowNbr[k], j + colNbr[k], visited):
        self.DFS(i + rowNbr[k], j + colNbr[k], visited) 
  def countIslands(self):
    visited = [[False for j in range(self.COL)]for i in range(self.ROW)] 
    count = 0
    for i in range(self.ROW):
      for j in range(self.COL):
        if visited[i][j] == False and self.graph[i][j] ==1:
          self.DFS(i, j, visited)
          count += 1
    return count

###**New Feature Calculation Using Region Calculation Program**

In [0]:
feat = []
for a in train_images:
  row = len(a)
  col = len(a[0])
  g = Graph(row, col, a)
  feat.append(g.countIslands())

###**Calculating Accuracy of Number of Regions Cauculated by our Program**

In [0]:
feat2 = [x for x in range(0,60000)]
for i in range(0,len(train_labels_raw)):
  if train_labels_raw[i] in (1,2,3,4,5,7):
    feat2[i] = 1
  elif train_labels_raw[i] in (0,6,9):
    feat2[i] = 2
  elif train_labels_raw[i] == 8:
    feat2[i] = 3
c = 0
for i in range(60000):
  if feat[i]== feat2[i]:
    c = c+1
(c/60000)*100.0
  

85.02499999999999

###**Image Reshaping and Normalization**

In [0]:
train_images_final = train_images_raw.reshape((60000, 28 * 28))
train_images_final = train_images_final.astype('float32') / 255

test_images_final = test_images_raw.reshape((10000, 28 * 28))
test_images_final = test_images_final.astype('float32') / 255

In [0]:
train_labels = to_categorical(train_labels_raw)
test_labels = to_categorical(test_labels_raw)

###**Appending New Feature into all images**

In [0]:
#For Train Images
xx = len(train_images_final)
yy = len(train_images_final[0])
tr_im = [[i for i in range(yy+1)] for j in range(xx)]
for i in range(0,len(train_images_final)):
  for j in range(0,len(train_images_final[i])):
    tr_im[i][j] = train_images_final[i][j]
  tr_im[i][yy] = feat[i]

In [0]:
#For Test Images
xx = len(test_images_final)
yy = len(test_images_final[0])
ts_im = [[i for i in range(yy+1)] for j in range(xx)]
for i in range(0,len(test_images_final)):
  for j in range(0,len(test_images_final[i])):
    ts_im[i][j] = test_images_final[i][j]
  ts_im[i][yy] = feat[i]

###** Converting List into Numpy Array**

In [0]:
tr_img = np.array(tr_im)
ts_img = np.array(ts_im)

###**Network Definition**

In [0]:
network = models.Sequential()
network.add(layers.Dense(10, activation='softmax'))

###**Compilation**

In [0]:
network.compile(optimizer='Adam',loss='categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


###**Training**

In [0]:
epochs = 100
history = network.fit(tr_img, 
                      train_labels, 
                      epochs=epochs, 
                      batch_size=128, 
                      validation_data=(ts_img, test_labels))
network.summary()

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 2s 37us/step - loss: 0.6800 - acc: 0.8327 - val_loss: 0.3997 - val_acc: 0.8972
Epoch 2/100
60000/60000 [==============================] - 2s 26us/step - loss: 0.3430 - acc: 0.9086 - val_loss: 0.3377 - val_acc: 0.9062
Epoch 3/100
60000/60000 [==============================] - 2s 26us/step - loss: 0.2952 - acc: 0.9198 - val_loss: 0.3223 - val_acc: 0.9098
Epoch 4/100
60000/60000 [==============================] - 2s 26us/step - loss: 0.2712 - acc: 0.9256 - val_loss: 0.3172 - val_acc: 0.9099
Epoch 5/100
60000/60000 [==============================] - 2s 26us/step - loss: 0.2567 - acc: 0.9297 - val_loss: 0.3226 - val_acc: 0.9061
Epoch 6/100
60000/60000 [==============================] - 2s 27us/step - loss: 0.2464 - acc: 0.9322 - val_loss: 0.3245 - val_acc: 0.9037
Epoch 7/100
60000/60000 [==============================] - 2s 28us/step - 

###**History**

In [0]:
history.history['acc']

[0.8326666666984558,
 0.9085666666984558,
 0.9198333333651225,
 0.9255666666348775,
 0.9297166666666666,
 0.93215,
 0.9333499999682109,
 0.9355666666666667,
 0.9367999999682108,
 0.9379999999682108,
 0.9387166666348775,
 0.9399,
 0.9410333333333334,
 0.9413333333651225,
 0.9419333333015442,
 0.9427833333333333,
 0.9434333333651225,
 0.9433666666984558,
 0.9443166666348776,
 0.9440833333651225,
 0.9446000000317891,
 0.9449833333333333,
 0.94515,
 0.9458666666984558,
 0.9459333333015442,
 0.9456000000317891,
 0.9462666666984558,
 0.9463500000317892,
 0.9465999999682109,
 0.9469999999682108,
 0.9473166666348776,
 0.9472333333651225,
 0.9472833333015442,
 0.94725,
 0.9474333333333333,
 0.948,
 0.9477666666666666,
 0.9480833333333333,
 0.9479166666984558,
 0.9485666666348775,
 0.9482833333015442,
 0.9479666666348775,
 0.9486833333333333,
 0.9485166666348775,
 0.9482333333015441,
 0.9490333333651225,
 0.9491833333651225,
 0.9492499999682109,
 0.94915,
 0.9495,
 0.9491500000317892,
 0.9495166